In [1]:
import sys
from nbfinder import NotebookFinder
sys.meta_path.append(NotebookFinder())
import tensorflow as tf
slim=tf.contrib.slim
if __name__ == "__main__":
    sys.path.append("../../../")

In [2]:
from notebooks_src.box_encode_decode.ssd.make_anchors_orig import make_anchors_for_one_fmap
from notebooks_src.box_encode_decode.ssd.encode import encode, encode_one_fmap
from notebooks_src.models.configs import configs
from notebooks_src.box_encode_decode.configs import configs as box_configs
from notebooks_src.losses.util import unpack_net_output, get_int_tensor_shape, sort_some_lists_of_tensors
from notebooks_src.losses.util import abs_smooth as smooth_L1
configs.update(box_configs)

importing Jupyter notebook from ../../../notebooks_src/box_encode_decode/ssd/make_anchors_orig.ipynb
importing Jupyter notebook from ../../../notebooks_src/box_encode_decode/configs.ipynb
importing Jupyter notebook from ../../../notebooks_src/box_encode_decode/ssd/encode.ipynb
importing Jupyter notebook from ../../../notebooks_src/losses/util.ipynb
importing Jupyter notebook from ../../../notebooks_src/models/configs.ipynb
importing Jupyter notebook from ../../../notebooks_src/config_util.ipynb
importing Jupyter notebook from ../../../notebooks_src/load_data/configs.ipynb
importing Jupyter notebook from ../../../notebooks_src/load_data/get_generator.ipynb
importing Jupyter notebook from ../../../notebooks_src/load_data/generator/generator.ipynb
importing Jupyter notebook from ../../../notebooks_src/box_encode_decode/util.ipynb
importing Jupyter notebook from ../../../notebooks_src/load_data/generator/batch_fetcher.ipynb


In [3]:
def compute_loss(y_true, y_preds):
    '''y_true: the boxes Nx15x5 tensor
       y_preds: a list of 7?  tensors of Nxfy x fx x k where k = 4*number of anchors + number_of_anchors*num_classes,
       N is number of examples'''

    gclasses, glocalizations, gscores = encode(y_true)
    
    localizations, logits, predictions = unpack_net_output(y_preds)
    
    #sort so in same order
    logits, localizations, gclasses, glocalizations, gscores = sort_some_lists_of_tensors(logits,
                                                                                           localizations, 
                                                                                           gclasses, 
                                                                                           glocalizations, 
                                                                                           gscores)

    final_loss = losses(logits, localizations,
           gclasses, glocalizations, gscores)
    return final_loss

In [4]:
def losses(logits, localisations,
           gclasses, glocalisations, gscores,
           match_threshold=0.5,
           negative_ratio=3.,
           alpha=1.,
           label_smoothing=0.,
           scope='ssd_losses'):
    """Define the SSD network losses.
    """
    

    final_loss = ssd_losses(logits, localisations,
                      gclasses, glocalisations, gscores,
                      match_threshold=match_threshold,
                      negative_ratio=negative_ratio,
                      alpha=alpha,
                      label_smoothing=label_smoothing,
                      scope=scope)
    return final_loss

In [5]:
def ssd_losses(logits, localisations,
               gclasses, glocalisations, gscores,
               match_threshold=0.5,
               negative_ratio=3.,
               alpha=1.,
               label_smoothing=0.,
               device='/cpu:0',
               scope=None):
    with tf.name_scope(scope, 'ssd_losses'):
        lshape = get_int_tensor_shape(logits[0])
        num_classes = lshape[-1]
        batch_size = lshape[0]

        # Flatten out all vectors!
        flogits = []
        fgclasses = []
        fgscores = []
        flocalisations = []
        fglocalisations = []
        for i in range(len(logits)):
            flogits.append(tf.reshape(logits[i], [-1, num_classes]))
            fgclasses.append(tf.reshape(gclasses[i], [-1]))
            fgscores.append(tf.reshape(gscores[i], [-1]))
            flocalisations.append(tf.reshape(localisations[i], [-1, 4]))
            fglocalisations.append(tf.reshape(glocalisations[i], [-1, 4]))
        # And concat the crap!
        logits = tf.concat(flogits, axis=0)
        gclasses = tf.concat(fgclasses, axis=0)
        gscores = tf.concat(fgscores, axis=0)
        localisations = tf.concat(flocalisations, axis=0)
        glocalisations = tf.concat(fglocalisations, axis=0)
        dtype = logits.dtype

        # Compute positive matching mask...
        pmask = gscores > match_threshold
        fpmask = tf.cast(pmask, dtype)
        n_positives = tf.reduce_sum(fpmask)

        # Hard negative mining...
        no_classes = tf.cast(pmask, tf.int32)
        predictions = slim.softmax(logits)
        nmask = tf.logical_and(tf.logical_not(pmask),
                               gscores > -0.5)
        fnmask = tf.cast(nmask, dtype)
        nvalues = tf.where(nmask,
                           predictions[:, 0],
                           1. - fnmask)
        nvalues_flat = tf.reshape(nvalues, [-1])
        # Number of negative entries to select.
        max_neg_entries = tf.cast(tf.reduce_sum(fnmask), tf.int32)
        n_neg = tf.cast(negative_ratio * n_positives, tf.int32) + batch_size
        n_neg = tf.minimum(n_neg, max_neg_entries)

        val, idxes = tf.nn.top_k(-nvalues_flat, k=n_neg)
        max_hard_pred = -val[-1]
        # Final negative mask.
        nmask = tf.logical_and(nmask, nvalues < max_hard_pred)
        fnmask = tf.cast(nmask, dtype)

        # Add cross-entropy loss.
        with tf.name_scope('cross_entropy_pos'):
            loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,
                                                                  labels=gclasses)
            loss = tf.div(tf.reduce_sum(loss * fpmask), batch_size, name='value')
            tf.losses.add_loss(loss)
            xent_pos = loss

        with tf.name_scope('cross_entropy_neg'):
            loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,
                                                                  labels=no_classes)
            loss = tf.div(tf.reduce_sum(loss * fnmask), batch_size, name='value')
            tf.losses.add_loss(loss)
            xent_neg = loss

        # Add localization loss: smooth L1, L2, ...
        with tf.name_scope('localization'):
            # Weights Tensor: positive mask + random negative.
            weights = tf.expand_dims(alpha * fpmask, axis=-1)
            loss = smooth_L1(localisations - glocalisations)
            loss = tf.div(tf.reduce_sum(loss * weights), batch_size, name='value')
            tf.losses.add_loss(loss)
            loc = loss
        return loc + xent_pos + xent_neg

In [6]:
if __name__ == "__main__":
    with tf.Session() as sess:
        from notebooks_src.load_data.get_generator import get_generator

        gen=get_generator("tr", batch_size=2)
        bboxes = tf.placeholder(tf.float32,shape=(2,15,5),name="bboxes")
        shapes = [(2, 6, 9, 48),
                 (2, 3, 5, 32),
                 (2, 96, 144, 32),
                 (2, 24, 36, 48),
                 (2, 12, 18, 48),
                 (2, 48, 72, 48),
                 (2, 1, 1, 32)]

        y_preds = [tf.ones((shape)) for shape in shapes]
        final_loss = compute_loss(y_true=bboxes,y_preds=y_preds)
        #final_loss = compute_loss(y_true=bboxes,y_preds=y_preds)

        
        for im, box in gen:
            print sess.run(final_loss, feed_dict={bboxes:box})

importing Jupyter notebook from ../../../notebooks_src/load_data/datasets/climate/get_data.ipynb
importing Jupyter notebook from ../../../notebooks_src/load_data/datasets/climate/util.ipynb
